# LSTM for Spoken Digit Classification

This notebook builds and trains a recurrent neural network (LSTM) to classify spoken digits (0–9) from audio recordings.

- Dataset: [Free Spoken Digit Dataset (FSDD)](https://github.com/Jakobovski/free-spoken-digit-dataset)
- Framework: PyTorch
- Architecture: RNN with LSTM layers

In [21]:
import sys
import os

# Add the project root (parent of current folder) to Python path
project_root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root_dir)


## Load Model Configuration from YAML

To make the training pipeline configurable and modular, we store model parameters like number of LSTM layers, hidden size, and learning rate etc in a YAML file. This structure enables quick adaptation to related tasks B, and C.

This section loads the model configuration using a custom utility function.

In [22]:
import src.utils as utils

In [23]:
utils.set_seed(42)

[INFO] Random seed set to: 42


In [24]:
import yaml
import json

model_config_path = os.path.join(project_root_dir, 'config', 'model_config.yaml')
model_config = utils.read_yaml_file(model_config_path)
# print(json.dumps(model_config, indent=2))

## Load and Split Dataset for Training and Evaluation

In this section, we load the recordings data from disk, generate data-label pairs, and split them into training and test sets according to the `test_size` defined in the YAML file.

Using `test_size` and `seed` from the YAML config ensures that experiments are reproducible and easily tunable for other tasks by simply updating the configuration.


In [25]:
data_path = model_config['dataset']['path']
test_data_size = model_config['data_splitting']['test_size']
seed = model_config['experiment']['seed']

In [26]:
data_label_pairs = utils.prepare_data_label_pairs(data_path)

In [27]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data_label_pairs, test_size=test_data_size, random_state=seed)

## Transform Raw Data into PyTorch Dataset Objects

The `AudioFeaturesDataset` class converts raw data-label pairs into PyTorch-compatible datasets that provide easy access to samples and labels.

AudioFeaturesDataset is a custom dataset class that:

- Loads audio recordings of spoken digits along with their labels.
- Optionally cleans the audio by filtering out noise.
- Extracts MFCC features (a common speech feature).
- Pads or trims these features to a fixed length so all inputs have the same shape.
- Works with PyTorch to provide samples one-by-one when training or testing a model.
- It helps prepare your audio data in the right format for training neural networks efficiently.


In [28]:
from src.data_preprocessor import AudioFeaturesDataset

train_dataset = AudioFeaturesDataset(train_data)
test_dataset = AudioFeaturesDataset(test_data)

In [29]:
print(f"Train size: {len(train_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 2400
Test size: 600


## Create DataLoaders for Batch Processing

Using PyTorch DataLoaders, we enable efficient loading, batching, and shuffling of data during training and evaluation.

In [30]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

## LSTM Model Definition

A simple `n`-layer LSTM followed by a fully connected output layer. Variable `n` is defined in the configuration YAML file

In [31]:
input_dim = model_config['model']['input_dim']
hidden_dim = model_config['model']['hidden_dim']
num_layers = model_config['model']['num_layers']
output_dim = model_config['model']['output_dim']

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
import src.model as model
import torch.nn as nn
import torch.optim as optim

model = model.LSTMClassifier(input_dim=input_dim,
                       hidden_dim=hidden_dim,
                       num_layers=num_layers,
                       output_dim=output_dim).to(device)

## Training Loop

In [34]:
learning_rate = model_config['training']['learning_rate']
epochs = model_config['training']['epochs']

In [15]:
from src.train import ModelTrainer
trainer_instance = ModelTrainer(
    model, 
    epochs,
    train_loader,
    device,
    learning_rate
)

In [16]:
trainer_instance.train()

Epoch [1/20], Loss: 134.0127, Accuracy: 57.67%
Epoch [2/20], Loss: 49.1018, Accuracy: 86.17%
Epoch [3/20], Loss: 21.3893, Accuracy: 96.75%
Epoch [4/20], Loss: 12.2558, Accuracy: 96.42%
Epoch [5/20], Loss: 9.3282, Accuracy: 98.12%
Epoch [6/20], Loss: 6.7593, Accuracy: 96.42%
Epoch [7/20], Loss: 5.2892, Accuracy: 98.58%
Epoch [8/20], Loss: 4.0651, Accuracy: 99.17%
Epoch [9/20], Loss: 1.7189, Accuracy: 99.33%
Epoch [10/20], Loss: 1.9559, Accuracy: 99.79%
Epoch [11/20], Loss: 3.8065, Accuracy: 95.79%
Epoch [12/20], Loss: 3.7462, Accuracy: 99.79%
Epoch [13/20], Loss: 1.1268, Accuracy: 99.79%
Epoch [14/20], Loss: 1.7250, Accuracy: 99.08%
Epoch [15/20], Loss: 2.5933, Accuracy: 99.62%
Epoch [16/20], Loss: 2.1036, Accuracy: 99.25%
Epoch [17/20], Loss: 1.4843, Accuracy: 99.67%
Epoch [18/20], Loss: 0.4845, Accuracy: 99.92%
Epoch [19/20], Loss: 0.8350, Accuracy: 100.00%
Epoch [20/20], Loss: 0.1467, Accuracy: 100.00%


In [17]:
_, _ = utils.get_model_params_size(model)


Layer-wise parameter counts:
lstm.weight_ih_l0              -> 6,656 params
lstm.weight_hh_l0              -> 65,536 params
lstm.bias_ih_l0                -> 512 params
lstm.bias_hh_l0                -> 512 params
lstm.weight_ih_l1              -> 65,536 params
lstm.weight_hh_l1              -> 65,536 params
lstm.bias_ih_l1                -> 512 params
lstm.bias_hh_l1                -> 512 params
fc.weight                      -> 1,280 params
fc.bias                        -> 10 params


 Total Parameters: 206,602
Estimated Memory: 807.04 KB (0.79 MB)


In [18]:
print(model)

LSTMClassifier(
  (lstm): LSTM(13, 128, num_layers=2, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


## Evaluation & Visualization

In [18]:
from src.evaluate import ModelEvaluator

In [19]:
test_instance = ModelEvaluator(
    model, 
    test_loader,
    device
)


In [21]:
test_instance.evaluate()


 Accuracy on test data: 99.17%

 Classification Report:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        52
           1     1.0000    1.0000    1.0000        65
           2     0.9692    1.0000    0.9844        63
           3     1.0000    0.9667    0.9831        60
           4     1.0000    1.0000    1.0000        55
           5     1.0000    0.9811    0.9905        53
           6     1.0000    0.9846    0.9922        65
           7     1.0000    0.9831    0.9915        59
           8     0.9683    1.0000    0.9839        61
           9     0.9853    1.0000    0.9926        67

    accuracy                         0.9917       600
   macro avg     0.9923    0.9915    0.9918       600
weighted avg     0.9919    0.9917    0.9917       600


 Confusion Matrix:
[[52  0  0  0  0  0  0  0  0  0]
 [ 0 65  0  0  0  0  0  0  0  0]
 [ 0  0 63  0  0  0  0  0  0  0]
 [ 0  0  1 58  0  0  0  0  1  0]
 [ 0  0  0  0 55  0  0  0  0  0]

{'accuracy': 99.16666666666667,
 'predictions': array([6, 6, 7, 5, 4, 5, 6, 2, 9, 7, 5, 4, 5, 1, 5, 1, 0, 3, 0, 3, 8, 3,
        7, 2, 1, 2, 9, 8, 6, 2, 8, 4, 5, 0, 4, 5, 4, 2, 8, 2, 0, 0, 6, 7,
        4, 6, 4, 2, 6, 8, 4, 6, 0, 1, 7, 6, 7, 9, 8, 3, 5, 9, 6, 0, 7, 9,
        9, 6, 0, 9, 8, 4, 5, 3, 8, 7, 1, 6, 1, 8, 5, 9, 8, 4, 4, 2, 5, 8,
        6, 7, 0, 5, 0, 4, 3, 1, 7, 1, 2, 5, 9, 4, 0, 8, 7, 7, 2, 8, 4, 2,
        2, 7, 9, 5, 9, 9, 9, 3, 0, 5, 9, 3, 9, 0, 8, 8, 5, 6, 2, 4, 5, 0,
        1, 6, 2, 1, 8, 9, 9, 4, 6, 5, 2, 8, 4, 9, 1, 0, 3, 8, 3, 0, 4, 4,
        2, 4, 4, 5, 9, 4, 6, 7, 7, 8, 4, 3, 7, 0, 3, 2, 7, 8, 0, 6, 7, 9,
        4, 2, 6, 7, 0, 6, 2, 6, 1, 3, 9, 4, 8, 3, 3, 4, 4, 5, 7, 2, 0, 1,
        3, 2, 4, 2, 2, 4, 5, 7, 0, 1, 2, 2, 9, 6, 9, 9, 0, 5, 9, 5, 6, 2,
        3, 6, 1, 4, 5, 3, 8, 1, 3, 4, 4, 9, 4, 6, 7, 0, 7, 6, 3, 9, 1, 5,
        6, 4, 9, 8, 7, 4, 6, 4, 3, 6, 3, 8, 8, 9, 8, 8, 9, 7, 9, 0, 5, 5,
        4, 3, 3, 0, 3, 2, 8, 4, 0, 8, 7, 8, 2, 1, 5, 8, 1, 4, 6, 

## Task B — Retrain Under Memory Constraints

- All parameters of any one layer must fit into memory simultaneously
- Maximum memory available for layer parameters is 36 KB

Since Pytorch stores all the layer parameters as floating point values, as 32-bit floats(4 bytes per parameter), this implies that the 
maximum number of parameters should not exceed

$$
\text{Max total number of parameters} = \frac{36\,\text{KB}}{4\,\text{bytes}} = \frac{36 \times 1024}{4} = 9,216 \text{ parameters}
$$

## Model Parameter Breakdown for 2 Layers LSTM
The following calculations are based on the parameter definitions from PyTorch's LSTM implementation, as described in the \href{https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html\#torch.nn.LSTM}{official documentation}. 

Let’s define the following variables:

$$
\begin{aligned}
\text{Input dimension} &= i \\
\text{Hidden dimension} &= h \\
\text{Output dimension} &= o \\
\text{Number of LSTM layers} &= 2 \\
\text{Fully Connected (Linear) layer count} &= 1 \\
\end{aligned}
$$

These will be used to calculate the total number of parameters in the model.

---

## LSTM Layer Parameters

Each LSTM layer has 4 internal gates (input, forget, cell, output).  
So each layer has:

$$
\begin{aligned}
W\_{ih} &: \text{Weights from input to hidden} \rightarrow \text{shape: } (4 \times h, i) \\
W\_{hh} &: \text{Weights from hidden to hidden} \rightarrow \text{shape: } (4 \times h, h) \\
b\_{ih}, b\_{hh} &: \text{Biases for each gate} \rightarrow \text{shape: } (4 \times h)
\end{aligned}
$$

### Calculations:

$$
\begin{aligned}
&\textbf{Layer 1 Parameters} \\
&W\_ih: 4 \times h \times i \\
&W\_hh: 4 \times h \times h \\
&b\_ih: 4 \times h \\
&b\_hh: 4 \times h \\
&\\
&\textbf{Layer 2 Parameters} \\
&W\_ih: 4 \times h \times h \\
&W\_hh: 4 \times h \times h \\
&b\_ih: 4 \times h \\
&b\_hh: 4 \times h \\
&\\
&\text{Total LSTM parameters} = (4 \times h \times i) + (4 \times h \times h) + (8 \times h) = 4hi + 4h^2 + 8h
\end{aligned}
$$

---

## Fully Connected (Linear) Layer

- Input features = $$\text{hidden\_dim} = h$$  
- Output features = $$\text{output\_dim} = o$$

### Calculations:
$$
\begin{aligned}
&\text{Weights}: \quad h \times o \\
&\text{Bias}: \quad o \\
&\textbf{Total Linear parameters} = h \cdot o + o = o(h + 1)
\end{aligned}
$$

---

## Total Parameters

The total number of parameters in the model is the sum of the LSTM and Linear layer parameters:

$$
\text{Total Parameters} = \text{Total LSTM Parameters} + \text{Total Linear Parameters}
$$

From earlier calculations:

- $\text{Total LSTM Parameters} = 4hi + 4h^2 + 8h$
- $\text{Total Linear Parameters} = h \cdot o + o = o(h + 1)$

Therefore:

$$
\text{Total Parameters} = (4hi + 4h^2 + 8h) + o(h + 1)
$$

Or more compactly:

$$
\boxed{\text{Total Parameters} = 4hi + 4h^2 + 8h + o(h + 1)}
$$

---


## Designing a 2-Layer LSTM Under a 36 KB Memory Constraint

Given an output dimension of $10$ (representing 10 classes or digits) and an input dimension of $13$ (corresponding to 13 MFCC coefficients per time step), the total number of parameters in the model reduces to the following quadratic expression:

$$
\text{Total Parameters} = 12h^2 + 78h + 10
$$

Here, $ h $ (the hidden dimension) remains the only variable we need to solve for.

Since the memory constraint allows for a maximum of \textbf{9,216 parameters}, the hidden dimension must satisfy:

$$
12h^2 + 78h + 10 \leq 9216
$$

The maximum valid integer value of $ h $ that satisfies the inequality is:
$$
h = 24
$$

---

In [38]:
import src.model as model_with_constraints

hidden_dim = 24
two_layer_model_with_constraints = model_with_constraints.LSTMClassifier(input_dim=input_dim,
                                           hidden_dim=hidden_dim,
                                           num_layers=num_layers,
                                           output_dim=output_dim).to(device)

In [41]:
from src.train import ModelTrainer
trainer_instance = ModelTrainer(
    two_layer_model_with_constraints, 
    epochs,
    train_loader,
    device,
    learning_rate
)

In [42]:
trainer_instance.train()

Epoch [1/20], Loss: 172.2554, Accuracy: 18.17%
Epoch [2/20], Loss: 141.0399, Accuracy: 32.17%
Epoch [3/20], Loss: 106.1623, Accuracy: 52.29%
Epoch [4/20], Loss: 90.3998, Accuracy: 59.21%
Epoch [5/20], Loss: 78.1598, Accuracy: 64.83%
Epoch [6/20], Loss: 69.3477, Accuracy: 73.33%
Epoch [7/20], Loss: 58.6390, Accuracy: 76.50%
Epoch [8/20], Loss: 51.9842, Accuracy: 79.42%
Epoch [9/20], Loss: 47.4075, Accuracy: 81.67%
Epoch [10/20], Loss: 38.8978, Accuracy: 84.08%
Epoch [11/20], Loss: 37.5306, Accuracy: 84.50%
Epoch [12/20], Loss: 35.1007, Accuracy: 87.12%
Epoch [13/20], Loss: 29.0342, Accuracy: 90.62%
Epoch [14/20], Loss: 26.8872, Accuracy: 91.21%
Epoch [15/20], Loss: 22.7544, Accuracy: 93.92%
Epoch [16/20], Loss: 19.4326, Accuracy: 94.58%
Epoch [17/20], Loss: 17.7522, Accuracy: 95.67%
Epoch [18/20], Loss: 15.5943, Accuracy: 96.08%
Epoch [19/20], Loss: 14.5663, Accuracy: 95.08%
Epoch [20/20], Loss: 13.6361, Accuracy: 96.79%


In [43]:
_, _ = utils.get_model_params_size(two_layer_model_with_constraints)


Layer-wise parameter counts:
lstm.weight_ih_l0              -> 1,248 params
lstm.weight_hh_l0              -> 2,304 params
lstm.bias_ih_l0                -> 96 params
lstm.bias_hh_l0                -> 96 params
lstm.weight_ih_l1              -> 2,304 params
lstm.weight_hh_l1              -> 2,304 params
lstm.bias_ih_l1                -> 96 params
lstm.bias_hh_l1                -> 96 params
fc.weight                      -> 240 params
fc.bias                        -> 10 params


 Total Parameters: 8,794
Estimated Memory: 34.35 KB (0.03 MB)


In [44]:
from src.evaluate import ModelEvaluator

In [45]:
test_instance = ModelEvaluator(
    two_layer_model_with_constraints, 
    test_loader,
    device
)

In [46]:
test_instance.evaluate()


 Accuracy on test data: 93.67%

 Classification Report:
              precision    recall  f1-score   support

           0     0.9800    0.9423    0.9608        52
           1     1.0000    0.9538    0.9764        65
           2     0.8857    0.9841    0.9323        63
           3     0.9811    0.8667    0.9204        60
           4     0.9821    1.0000    0.9910        55
           5     0.8966    0.9811    0.9369        53
           6     0.8889    0.8615    0.8750        65
           7     0.9825    0.9492    0.9655        59
           8     0.8507    0.9344    0.8906        61
           9     0.9531    0.9104    0.9313        67

    accuracy                         0.9367       600
   macro avg     0.9401    0.9384    0.9380       600
weighted avg     0.9394    0.9367    0.9369       600


 Confusion Matrix:
[[49  0  2  0  0  0  0  1  0  0]
 [ 0 62  1  0  0  2  0  0  0  0]
 [ 0  0 62  0  0  0  0  0  0  1]
 [ 0  0  3 52  0  0  2  0  2  1]
 [ 0  0  0  0 55  0  0  0  0  0]

## Floating Point Restriction (Task B Continued)

In addition to the 36 KB memory constraint, the hardware for Task B is limited in computational capability and **does not support floating point operations**. It means the network must operate using **integer or fixed-point arithmetic** only. **Weights, activations, and computations** should be **quantized** to lower-precision formats such as 8-bit integers (INT8)


In [54]:
two_layer_model_with_constraints_cpu = two_layer_model_with_constraints.to('cpu') 
# Apply dynamic quantization to the entire model or just LSTM/Linear layers
quantized_model = torch.quantization.quantize_dynamic(
    two_layer_model_with_constraints_cpu,  # the model instance
    {nn.LSTM, nn.Linear},  # layers to quantize
    dtype=torch.qint8  # quantize to 8-bit integers
)

In [56]:
from src.evaluate import ModelEvaluator

In [57]:
device = torch.device("cpu")
test_instance = ModelEvaluator(
    quantized_model, 
    test_loader,
    device
)

In [58]:
test_instance.evaluate()


 Accuracy on test data: 93.50%

 Classification Report:
              precision    recall  f1-score   support

           0     0.9800    0.9423    0.9608        52
           1     1.0000    0.9385    0.9683        65
           2     0.8732    0.9841    0.9254        63
           3     0.9811    0.8667    0.9204        60
           4     0.9821    1.0000    0.9910        55
           5     0.8814    0.9811    0.9286        53
           6     0.8889    0.8615    0.8750        65
           7     0.9821    0.9322    0.9565        59
           8     0.8507    0.9344    0.8906        61
           9     0.9688    0.9254    0.9466        67

    accuracy                         0.9350       600
   macro avg     0.9388    0.9366    0.9363       600
weighted avg     0.9385    0.9350    0.9353       600


 Confusion Matrix:
[[49  0  2  0  0  0  0  1  0  0]
 [ 0 61  1  0  0  3  0  0  0  0]
 [ 0  0 62  0  0  0  0  0  0  1]
 [ 0  0  4 52  0  0  2  0  2  0]
 [ 0  0  0  0 55  0  0  0  0  0]

## Total Parameters for 1-Layer LSTM

For a single LSTM layer, with the same input and output dimensions (13 MFCC coefficients and 10 output classes), the total number of parameters simplifies to:

$$
\text{Total Parameters} = 4h^2 + 70h + 10
$$